In [1]:
from fastapi import FastAPI
from bs4 import BeautifulSoup
import requests
import uvicorn
import asyncio
import json
import pandas as pd
import numpy as np
from io import StringIO  
from typing import List, Dict

In [2]:
app = FastAPI()

In [3]:
@app.get("/")
async def test():
    return {"message": "Hello World"}

In [4]:
@app.get("/llm_benchmarks/trustbit")
async def get_llm_benchmark_data_from_trustbit():
    base_url = 'https://www.trustbit.tech'
    html = requests.get(f'{base_url}/en/llm-benchmarks').text
    soup = BeautifulSoup(html, 'html.parser')
    
    links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith('/en/llm-leaderboard')]
    u_links = list(set(links))
    urls = [base_url + uri for uri in links]
    
    data = []
    for url in urls:
        response = requests.get(url)
    
        benchmark_html = response.text
    
        soup = BeautifulSoup(benchmark_html, 'html.parser')
        html_table = soup.find('table', class_='custom-model-table')
        if html_table == None: continue
        dfs = pd.read_html(StringIO(str(html_table)))
        # dfs is a list of DataFrames, one for each table found in the HTML
        if dfs:
            df = dfs[0]  # Assuming you want the first table

        title = url.split("-")[-2:]
        df.replace([np.nan, float('inf'), float('-inf')], None, inplace=True)
        df['leaderboard'] = "-".join(title)

        # Print the DataFrame
        data.append(df.to_dict(orient='records'))
    
    return {"data": data}

In [5]:
base_url = 'https://www.trustbit.tech'
html = requests.get(f'{base_url}/en/llm-benchmarks').text
soup = BeautifulSoup(html, 'html.parser')

links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith('/en/llm-leaderboard')]
u_links = list(set(links))
urls = [base_url + uri for uri in links]

data = []
for url in urls:
    response = requests.get(url)

    benchmark_html = response.text

    soup = BeautifulSoup(benchmark_html, 'html.parser')
    html_table = soup.find('table', class_='custom-model-table')
    if html_table == None: continue
    dfs = pd.read_html(StringIO(str(html_table)))

    # dfs is a list of DataFrames, one for each table found in the HTML
    if dfs:
        df = dfs[0]  # Assuming you want the first table
    title = url.split("-")[-2:]
    # Print the DataFrame
    df.replace([np.nan, float('inf'), float('-inf')], None, inplace=True)
    df['leaderboard'] = "-".join(title)
    print(df)

    data.append(df.to_dict(orient='records'))
    

for d in data:
    print(d)
    


                                        model  code   crm   docs integrate  \
0                                        None  None  None   None      None   
1                                   GPT-4o ☁️  90.0  95.0  100.0      90.0   
2                GPT-4 Turbo v5/2024-04-09 ☁️  86.0  99.0   98.0      93.0   
3               Google Gemini Pro 1.5 0801 ☁️  84.0  92.0   90.0     100.0   
4                            GPT-4 v1/0314 ☁️  90.0  88.0   98.0      52.0   
..                                        ...   ...   ...    ...       ...   
72  Microsoft Phi 3 Medium 4K Instruct f16 ⚠️   5.0  34.0   30.0      13.0   
73                           Orca 2 7B f16 ⚠️  22.0   0.0   24.0      18.0   
74                  Google Gemma 7B IT f16 ⚠️   0.0   0.0    0.0       9.0   
75                        Meta Llama2 7B f16🦙   0.0   5.0   18.0       3.0   
76                      Yi 1.5 9B Chat f16 ⚠️   0.0   4.0   29.0       8.0   

   marketing reason final 🏆    Cost     Speed leaderboard  
0  

In [6]:
if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())

INFO:     Started server process [16300]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
